In [ ]:
# Cayden Wagner

In [ ]:
%matplotlib widget
from urllib.request import urlopen
import json
import ipywidgets
import matplotlib.pyplot as plt

widgets = []
yARI = []
yFKG = []
xVals = []
graph = plt.subplot()
push = 0
reverse = False

def printBooks(item = None):
    global push
    global reverse
    string = "Books\n\n"
    
    if None == item:
        push = 0
    elif item.description == 'Sort: Forward':
        reverse = False
        push = 0
    elif item.description == 'Sort: Reverse':
        reverse = True
        push = len(xVals) - 1
        
    if not reverse and None != item:
        if item.description == 'top':
            push = 0
        elif item.description == 'bottom':
            push = len(bookData) - 10
        elif item.description == 'up':
            push += 10
            if push+10 > len(bookData):
                push = 0
            push = push % len(bookData)
        elif item.description == 'down':
            push -= 10 
            if push < 0:
                push = len(bookData) + push
    elif reverse and None != item:
        if item.description == 'top':
            push = len(xVals) - 1
        elif item.description == 'bottom':
            push = 9
        elif item.description == 'up':
            push -= 10
            if push < 0:
                push = len(bookData) + push
        elif item.description == 'down':
            push += 10
            push = push % len(bookData)
            
    if reverse:
        for i in range (push, push - 10, -1):
            string += f'{i+1}. {bookData[i]["bibliography"]["title"]}\n'
    else:
        for i in range (push, 10 + push):
            string += f'{i+1}. {bookData[i]["bibliography"]["title"]}\n'
            
    textBox.rows=string.count('\n')
    textBox.value=string
    
    buildGraph(push)
    
def buildGraph(start = 0):
    global graph
    global reverse
    graph.clear()
    [l.remove() for l in graph.lines]
    
    if reverse:
        minARI = min(yARI[start-9:start])
        maxARI = max(yARI[start-9:start])
        minFKG = min(yFKG[start-9:start])
        maxFKG = max(yFKG[start-9:start])

        graph.set_xlim(xVals[start], xVals[start-9])
        graph.set_ylim(min(minARI,minFKG)-2, max(maxARI,maxFKG)+2)
        
        graph.plot(xVals[start-9:start+1], yARI[start-9:start+1], label='ARI')
        graph.plot(xVals[start-9:start+1], yFKG[start-9:start+1], label='FKG')
        
        avgS.value = str(sum(yARI[start-10:start]) / 10)
    else:
        minARI = min(yARI[start:start+10])
        maxARI = max(yARI[start:start+10])
        minFKG = min(yFKG[start:start+10])
        maxFKG = max(yFKG[start:start+10])

        graph.set_xlim(xVals[start], xVals[start+9])
        graph.set_ylim(min(minARI,minFKG)-2, max(maxARI,maxFKG)+2)
    
        graph.plot(xVals[start:start+10], yARI[start:start+10], label='ARI')
        graph.plot(xVals[start:start+10], yFKG[start:start+10], label='FKG')
        avgS.value = str(sum(yARI[start:start+10]) / 10)
    
    graph.legend()
    
with open('data/bookData.json', encoding='utf-8') as inFile:
    bookData = json.load(inFile)
        
for i in range(len(bookData)):
    yARI.append(bookData[i]['metrics']['difficulty']['automated readability index'])
    yFKG.append(bookData[i]['metrics']['difficulty']['flesch kincaid grade'])
    xVals.append(i)

graph.set_ylabel("ARI/FKG")
graph.set_xlabel("Rank")
graph.set_title("ARI, FKG vs Rank")

top_btn = ipywidgets.Button(description = 'top')
bottom_btn = ipywidgets.Button(description = 'bottom')
down_btn = ipywidgets.Button(description = 'down')
up_btn = ipywidgets.Button(description = 'up')
SortF_btn = ipywidgets.Button(description = 'Sort: Forward')
SortR_btn = ipywidgets.Button(description = 'Sort: Reverse')
textBox = ipywidgets.Textarea(value='',
                              rows=1,
                              disabled=True, 
                              layout=ipywidgets.Layout(height='100%', width="100%"))

avgAll = ipywidgets.Text(value=str(sum(yARI) / len(yARI)), description='Avg ARI (All):', disabled=True)
avgS = ipywidgets.Text(value='', description='Avg ARI (Sample):', disabled=True)

top_btn.on_click(printBooks)
bottom_btn.on_click(printBooks)
up_btn.on_click(printBooks)
down_btn.on_click(printBooks)
SortF_btn.on_click(printBooks)
SortR_btn.on_click(printBooks)
printBooks()

out = ipywidgets.Output()
with out:
    plt.show(graph)

widgets.append(ipywidgets.HBox( [top_btn, bottom_btn, down_btn, up_btn] ))
widgets.append(ipywidgets.HBox( [SortF_btn, SortR_btn] ))
widgets.append(ipywidgets.HBox( [textBox] ))
widgets.append(ipywidgets.HBox( [out] ))
widgets.append(ipywidgets.HBox( [avgAll] ))
widgets.append(ipywidgets.HBox( [avgS] ))

ipywidgets.VBox(widgets)